In [1]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

ModuleNotFoundError: No module named 'mass_spring'

In [ ]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

mA = mss.add(Mass(1, (1, 0, 0)))
mB = mss.add(Mass(2, (2, 0, 0)))
f1 = mss.add(Fix((0, 0, 0)))

# Replace springs with distance constraints

def distance(p, q):
    return ((p[0]-q[0])**2 + (p[1]-q[1])**2 + (p[2]-q[2])**2) ** 0.5

L1 = distance(mss.fixes[f1.nr].pos, mss.masses[mA.nr].pos)
L2 = distance(mss.masses[mA.nr].pos, mss.masses[mB.nr].pos)

dc1 = DistanceConstraint(f1, mA, L1)
dc2 = DistanceConstraint(mA, mB, L2)

mss.addDistanceConstraint(dc1)
mss.addDistanceConstraint(dc2)


In [ ]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(
            SphereBufferGeometry(0.2, 16, 16),
            MeshStandardMaterial(color='red'),
            position=m.pos
        )
    )

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(
            SphereBufferGeometry(0.2, 32, 16),
            MeshStandardMaterial(color='blue'),
            position=f.pos
        )
    )

springpos = []
for c in mss.constraints:
    pA = mss.fixes[c.c1.nr].pos if c.c1.type == 1 else mss.masses[c.c1.nr].pos
    pB = mss.fixes[c.c2.nr].pos if c.c2.type == 1 else mss.masses[c.c2.nr].pos
    springpos.append([pA, pB])

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)

axes = AxesHelper(1)


In [ ]:
view_width = 600
view_height = 400

camera = PerspectiveCamera(position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer


In [ ]:
from time import sleep

for i in range(10000):
    mss.simulate(0.02, 100)

    # update mass positions
    for m, mvis in zip(mss.masses, masses):
        mvis.position = tuple(m.pos)

    # update constraint lines
    springpos = []
    for c in mss.constraints:
        pA = mss.fixes[c.c1.nr].pos if c.c1.type == 1 else mss.masses[c.c1.nr].pos
        pB = mss.fixes[c.c2.nr].pos if c.c2.type == 1 else mss.masses[c.c2.nr].pos
        springpos.append([pA, pB])

    springs.geometry = LineSegmentsGeometry(positions=springpos)

    sleep(0.01)
